In [1]:
import pandas as pd
import numpy as np
import folium
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv ('stadium.csv')
df.head()

,Stadium Code,Name,City,State,Capacity,Surface,Year Opened
0,1,Jordan-Hare Stadium,Auburn,AL,87451,Grass,1939
1,2,Legion Field,Birmingham,AL,71594,FieldTurf,1927
2,3,Ladd-Peebles Stadium,Mobile,AL,40646,FieldTurf,1948
3,4,Movie Gallery Veterans Stadium,Troy,AL,30000,AstroPlay,1950
4,5,Bryant-Denny Stadium,Tuscaloosa,AL,101821,Grass,1929


In [3]:
# empty list for lat and long columns
longitude = []
latitude = []
   
# function for coordinates
def findGeocode(city):
        
    try:

        geolocator = Nominatim(user_agent="config.js")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)    
    
for i in (df["City"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [4]:
#add lat/long to data frame
df["Longitude"] = longitude
df["Latitude"] = latitude
  
df.head()

,Stadium Code,Name,City,State,Capacity,Surface,Year Opened,Longitude,Latitude
0,1,Jordan-Hare Stadium,Auburn,AL,87451,Grass,1939,-85.480783,32.609857
1,2,Legion Field,Birmingham,AL,71594,FieldTurf,1927,-1.902691,52.479699
2,3,Ladd-Peebles Stadium,Mobile,AL,40646,FieldTurf,1948,-88.043054,30.694357
3,4,Movie Gallery Veterans Stadium,Troy,AL,30000,AstroPlay,1950,26.238017,39.957374
4,5,Bryant-Denny Stadium,Tuscaloosa,AL,101821,Grass,1929,-87.567526,33.209561


In [5]:
# storing the data in JSON format
df.to_json('file.geojson', orient = 'split', compression = 'infer', index = 'true')
 
# reading the JSON file
df = pd.read_json('file.geojson', orient ='split', compression = 'infer')
 
# displaying the DataFrame
print(df)
df.dtypes


     Stadium Code                            Name        City State  Capacity  \
0               1             Jordan-Hare Stadium      Auburn    AL     87451   
1               2                    Legion Field  Birmingham    AL     71594   
2               3            Ladd-Peebles Stadium      Mobile    AL     40646   
3               4  Movie Gallery Veterans Stadium        Troy    AL     30000   
4               5            Bryant-Denny Stadium  Tuscaloosa    AL    101821   
..            ...                             ...         ...   ...       ...   
159           139                   Husky Stadium     Seattle    WA     72500   
160           141            Camp Randall Stadium     Madison    WI     79500   
161           142            Joan Edwards Stadium  Huntington    WV     38016   
162           143               Mountaineer Field  Morgantown    WV     63500   
163           144            War Memorial Stadium     Laramie    WY     33500   

       Surface  Year Opened

Stadium Code      int64
Name             object
City             object
State            object
Capacity          int64
Surface          object
Year Opened       int64
Longitude       float64
Latitude        float64
dtype: object

In [6]:
dfloc = df[['Latitude', 'Longitude']].copy()
dfloc.head()

,Latitude,Longitude
0,32.609857,-85.480783
1,52.479699,-1.902691
2,30.694357,-88.043054
3,39.957374,26.238017
4,33.209561,-87.567526


In [7]:
#list of lat/long
locations = dfloc[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)


164

In [75]:
#create map
m = folium.Map(location=[39.5, -98.5], zoom_start=4)
for point in range(0, len(df)):
    surface = df['Surface'].iloc[point]
    if surface == 'Grass':
        color = 'green'
    elif surface == 'FieldTurf':
        color = 'blue'
    elif surface == 'AstroPlay':
        color = 'purple'
    else:
        color = 'red'
    labels = df['Name'].iloc[point]
    folium.Marker(locationlist[point], popup=labels,
                 icon=folium.Icon(color=color, icon='university', prefix='fa')).add_to(m)
map

map

In [76]:
from folium.plugins import FloatImage
image_file = 'image.PNG'

FloatImage(image_file, bottom=0, left=86).add_to(m)

In [77]:
#generate map
m.save('map.html')